<a href="https://colab.research.google.com/github/joseemaro/bdm_datasets/blob/main/tpfinal_pre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## SE IMPORTAN LAS LIBRERIAS Y LOS DATASETS.

In [ ]:
!wget https://raw.githubusercontent.com/joseemaro/bdm_datasets/main/googleplaystore.csv

!wget https://raw.githubusercontent.com/joseemaro/bdm_datasets/main/googleplaystore_user_reviews.csv

import numpy as np # algebra linear
import pandas as pd # procesamiento de datos, CSV file 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')
from pylab import rcParams
# figure size in inches
%matplotlib inline
plt.style.use('seaborn-white')

## SE LEE EL DATASET CON LOS DATOS DE LAS APLICACIONES

In [18]:
# Lee el archivo
data = pd.read_csv("googleplaystore.csv", encoding = "utf-8") 

# PREPROCESAMIENTO DE LOS DATOS

In [ ]:
#se verifica si hay nan de los registros
data.isna().sum()

Se observa que el campo rating tiene una gran cantidad de faltantes, mientras que los campos type,content rating, current ver, y android ver tienen pocos faltantes.

In [ ]:
#se remplaza los nan de rating por la media debido a que son muchos registros
data['Rating'].fillna((data['Rating'].mean()), inplace=True)

data.isna().sum()

In [ ]:
#el resto de los registros con nan al ser una cantidad minima se borran
df1 = data.dropna()

df1.isna().sum()

In [ ]:
#SE VERIFICA SI HAY REGISTROS DUPLICADOS
df1[df1.duplicated(['App'])]

In [23]:
#se eliminan los duplicados basandose en el nombre de la app
df2 = df1.drop_duplicates(keep='last',subset=['App'])

In [24]:
#campo installs
#se limpian los caracteres para pasar el campo a entero
df2.Installs = df2.Installs.str.replace('+' , '')
df2.Installs = df2.Installs.str.replace(',' , '')

In [ ]:
#se convierte el campo  installs a int
df2.Installs = df2.Installs.astype('float').astype('int')
df2['Installs'].unique()

In [ ]:
#se convierte el campo price a float
df2['Price'] = df2['Price'].apply(lambda x: float(x.replace("$","")))
df2['Price'].unique()

In [ ]:
#se convierte el conteo de reviews a int
df2['Reviews']= df2['Reviews'].apply(lambda x: int(x))
df2['Reviews'].unique()

In [ ]:
#vamos a remplazar el varies with device del campo size por nan
df2['Size'].replace('Varies with device',np.nan,inplace=True)
#se convierte size a float
df2['Size']=df2['Size'].str.replace('M','e+6').str.replace('k','e+3').str.replace('Varies with device','0').astype('float')
df2['Size'].unique()

# VISUALIZACION DE DATOS


In [ ]:
#Numero de aplicaciones por categoria

fig = plt.figure(figsize=(16,8)) 
df2.groupby('Category').count()["App"].sort_values(ascending=False).plot(kind='bar',title='Numero de apps/Categoria', fontsize=10)
plt.ylabel('Cantidad') 

In [ ]:
#apps con 1 billon de descargas

dfa = df2.groupby(['Category' ,'App'])['Installs'].sum().reset_index()

dfa = dfa.sort_values('Installs', ascending = False)

dftop = dfa[dfa.Installs == 1000000000 ]


dftop.style.set_properties(**{'background-color': 'white',
                            'color': 'black',
                            'border-color': 'black'})

La lista de apps que tienen mas de 1 billon de descargas muestra a las siguientes aplicaciones:
Google ,Google Drive, Google Chrome Fast Secure , Google+ ,Google Photos ,Google Play Games ,Google Play Books ,Google News ,Hangouts

Google ,Google Drive, Google Chrome Fast Secure , Google+ ,Google Photos ,Google Play Games ,Google Play Books ,Google News ,Hangouts

Gmail ,Google Street View ,Google Play Movies & TV ,Maps - Navigate & Explore , YouTube (Apps pertenecientes a Google)

Facebook ,WhatsApp Messenger , Instagram ,Messenger – Text and Video Chat for Free (Apps pertenecientes a Facebook)

Skype - free IM & video calls (pertenece a Microsoft)

Subway Surfers (pertenece a SYBO GAMES)

La razon por la que las aplicaciones de google tienen mas descargas que el resto es porque vienen preinstaladas en todos los sistemas android.


In [ ]:
#Precio promedio de las apps en cada categoria
fig = plt.figure(figsize=(16,8)) 
df2.groupby('Category').mean().sort_values(by='Price',ascending='False')['Price'].plot(kind='bar',title='Price in USD', fontsize=10)
plt.ylabel('Precio promedio - USD', fontsize=15) 

In [ ]:
#Tamaño promedio de las aplicaciones
print("Mean size " + str(df2["Size"].mean()/(1024*1024)) + " Mb") #converting in MB
print("Heaviest app size " + str(df2["Size"].max()/(1024*1024)) + " Mb") #converting in MB
print("Smalles app size " + str(df2["Size"].min()/(1024*1024)) + " Mb") #converting in MB

In [ ]:
#Numero de descargas por categoria
fig = plt.figure(figsize=(16,8)) 
df2.groupby('Category').sum().sort_values(by='Installs',ascending='False')['Installs'].plot(kind='bar',title='Descargas por categoria', fontsize=10)
plt.ylabel('Numero de descargas', fontsize=15) 

In [ ]:
#Porcentaje de apps pagas y gratis
#verificar apps gratis
df3= df2.copy()
def is_free(price):
    if price > 0:
        return 1
    else:
        return 0
df3["is_free"] = df3["Price"].apply(lambda x: int(is_free(x))) 
fig = plt.figure(figsize=(16,8)) 
labels = ['0 = Free','1 = Paid']
sizes = df3['is_free'].value_counts(sort = True)
plt.pie(sizes,labels=labels,autopct='%1.1f%%', shadow=True,textprops={'fontsize': 16})
plt.title('Apps Gratis vs Pagas',size = 20)
plt.legend()
plt.show()

In [ ]:
#vamos a observar la distribucion de la variable rating con un histograma
fig = plt.figure(figsize=(16,8)) 
plt.hist(df2['Rating'],edgecolor = 'black', log=True)
plt.title("Distribucion del campo Rating", fontsize=15)
plt.xlabel('Rating', fontsize=15)
plt.tight_layout()
plt.legend()

In [ ]:
#tambien vamos a observar la distribucion de los precios
fig = plt.figure(figsize=(16,8)) 
plt.hist(df2['Price'],edgecolor = 'black', log=True)
plt.title("Distribucion del campo Price",  fontsize=15)
plt.xlabel('Price',  fontsize=15)
plt.tight_layout()
plt.legend()

In [ ]:
#mayor cantidad de reviews
most_no_of_reviews = df2.groupby('App')[['Reviews']].mean().sort_values('Reviews', ascending=False).head(10)

sns.barplot(most_no_of_reviews.index, most_no_of_reviews.Reviews)

plt.title('Top 10 Apps con mas reviews')
plt.xticks(rotation=100);
plt.ylabel('Reviews');
plt.xlabel('Category');

# Feature engineering - vamos a crear una columna que asocia descargas y precio



In [ ]:

df_paid_apps = df2[df2['Price']>0]
df_paid_apps["Ganancia"] = df_paid_apps["Installs"] * df_paid_apps["Price"]
#histograma de las aplicaciones una vez eliminadas las gratis
fig = plt.figure(figsize=(16,8)) 
plt.hist(df_paid_apps['Price'],edgecolor = 'black', log=True)
median_age = df_paid_apps['Price'].median()
plt.title("Precio de los apps")
plt.xlabel('Price')
plt.ylabel('Count')
plt.tight_layout()
plt.legend()

In [ ]:
#apps paga con mas ganancia
df_paid_apps.sort_values(by=['Ganancia'])

la app paga con mayor ganancia es minecraft con 69900000.0 usd.

In [ ]:
#ganancia por categoria
fig = plt.figure(figsize=(16,8)) 
df_paid_apps.groupby('Category').sum().sort_values(by='Ganancia',ascending='False')['Ganancia'].plot(kind='bar',title='Ganancia', fontsize=10)
plt.ylabel('Ganancia de descargas', fontsize=15)


#Predecir el rating de una aplicacion usando regresion linear

In [107]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X=df2[["Reviews","Price"]]
y=df2.Rating

X_train,X_test,y_train,y_test= train_test_split(X,y,random_state=42)

# se pasan los features a la misma escala
scaler=StandardScaler()
X= scaler.fit_transform(X)

from sklearn.linear_model import LinearRegression
lin_r= LinearRegression()
model= lin_r.fit(X_train,y_train)

rating= model.predict(np.array([[1000,3]]))
print("Predicted rating is:",rating)

Predicted rating is: [4.16738455]


In [108]:
y=model.intercept_ +(1000*model.coef_[0]+2*model.coef_[1])
print("Rating is:",y)

Rating is: 4.168012452372769


In [109]:
pred= model.predict(X_test)
pred

array([4.16925362, 4.16929188, 4.16925104, ..., 4.17373128, 4.16925394,
       4.16925037])

In [110]:
#test
from sklearn.metrics import mean_squared_error
print("RMSE: {}".format(np.sqrt(mean_squared_error(y_test,pred))))

RMSE: 0.4877243124257784
